# SNPs to Networks 2: KEGG

> Parsing gene entries for *Zea mays* from the KEGG database (Previously downloaded).

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# import tqdm
import os, re
import pickle as pkl

from dlgwas.core import *

## Load *Zea mays* gene list

In [ ]:
kegg_list_zma = mkdf_kegg_species_list(kegg_species_list = get_kegg_species_list(species = 'zma'))

In [ ]:
kegg_list_zma.head()

,gene,seq_type,chromosome_positon,gene_type
0,zma:103644366,CDS,1:34607..40208,uncharacterized protein LOC103644366
1,zma:100382519,CDS,1:complement(41254..46100),uncharacterized protein LOC100382519
2,zma:103649349,CDS,1:complement(92293..107876),protein FAR1-RELATED SEQUENCE 5-like
3,zma:115032971,CDS,1:complement(187229..189591),uncharacterized protein LOC115032971
4,zma:103630223,CDS,1:complement(200404..203170),"granule-bound starch synthase 1b, chloroplasti..."


## Read KEGG gene entries

###  Convert a single file into a dictionary
The gene files contain sections (ENTRY, NAME, AASEQ, etc.). To aid processing each of these sections will be separated into a key in a dictionary.

In [ ]:
kegg_file = 'zma_100125650.txt'
# convert back to KEGG format
kegg_gene_name = kegg_file.replace('_', ':').replace('.txt', '')

# convert back to kegg format
r_text = read_kegg_gene(kegg_gene = kegg_gene_name)

# parsing gene entry
r_text_list = r_text.split('\n')
r_text_list[0:5]

['ENTRY       100125650         CDS       T01088',
 'NAME        (RefSeq) barren inflorescence 2',
 'ORGANISM    zma  Zea mays (maize)',
 'POSITION    1:177940567..177942323',
 'MOTIF       Pfam: Pkinase PK_Tyr_Ser-Thr Kinase-like Pkinase_fungal']

In [ ]:
# Some files may not contain all sections so this list needs to be created for each file
section_names = [r_text_list[i][0:12].strip() for i in range(len(r_text_list)) ]
section_names = [e for e in section_names if e != '']

section_starts = [
    [i for i in range(len(r_text_list)) if re.match('^'+e, r_text_list[i])][0] 
    for e in section_names]

section_names, section_starts

(['ENTRY',
  'NAME',
  'ORGANISM',
  'POSITION',
  'MOTIF',
  'DBLINKS',
  'AASEQ',
  'NTSEQ',
  '///'],
 [0, 1, 2, 3, 4, 5, 8, 18, 44])

In [ ]:
# just split each file into text of it's sections
out = {}

# get lines associated with section
def _get_section_text(section_name = 'AASEQ'):
    idx = section_names.index(section_name)
    section_text = r_text_list[section_starts[idx]:section_starts[idx+1]]
    # remove leading indent
    section_text = [e[12:] for e in section_text]
    return(section_text)

for section_name in section_names:
    if section_name != '///': # end of file
        out[section_name] = _get_section_text(section_name = section_name)

In [ ]:
out

{'ENTRY': ['100125650         CDS       T01088'],
 'NAME': ['(RefSeq) barren inflorescence 2'],
 'ORGANISM': ['zma  Zea mays (maize)'],
 'POSITION': ['1:177940567..177942323'],
 'MOTIF': ['Pfam: Pkinase PK_Tyr_Ser-Thr Kinase-like Pkinase_fungal'],
 'DBLINKS': ['NCBI-GeneID: 100125650',
  'NCBI-ProteinID: NP_001106051',
  'UniProt: A6MW92'],
 'AASEQ': ['491',
  'MDAAVRVPPALGNKTVTEVTPPPPPPAGEERLSDADTTASSTAAPNSSLSSASSAASLPR',
  'CSSLSRLSFDCSPSAALSSSSAAAAAAAASSPAPAPARPHRAGDAAWAAIRAASASAAAP',
  'LGPRDFRLLRRVGGGDVGTVYLCRLRAPPAPAPVCCLYAMKVVDRRVAAAKKKLEHAAAE',
  'RRILRALDHPFLPTLFADFDAAPHFSCVVTEFCPGGDLHSLRHRMPNRRFPLPSARFYAA',
  'EVLLALEYLHMMGIVYRDLKPENVLIRADGHIMLTDFDLSLQCTSTPSLEPCAAPEAAAA',
  'SCFPDHLFRRRRARLRRAASARRPPTTLVAEPVEARSCSFVGTHEYVAPEVARGGPHGAA',
  'VDWWALGVFLYELLHGRTPFAGADNEATLRNIARRPLSFPAAGAGDADARDLIARLLAKD',
  'PRHRLGSRRGAADVKAHPFFRGLNFALLRSSRPPVVPAASRSPLHRSQSCSAARTRASKP',
  'KPPPDTRFDLF'],
 'NTSEQ': ['1476',
  'atggacgccgcggtgcgcgtccccccggcgctcgggaacaagacggtgaccgaggtgacg',
  'ccgcc

### Convert all downloaded gene files

In [ ]:
def kegg_gene_to_dict(kegg_file = 'zma_100125650.txt'):
    # convert back to kegg format
    r_text = read_kegg_gene(kegg_gene= kegg_file.replace('_', ':').replace('.txt', '') )

    # parsing gene entry
    r_text_list = r_text.split('\n')

    # Some files may not contain all sections so this list needs to be created for each file
    section_names = [r_text_list[i][0:12].strip() for i in range(len(r_text_list)) ]
    section_names = [e for e in section_names if e != '']

    section_starts = [
        [i for i in range(len(r_text_list)) if re.match('^'+e, r_text_list[i])][0] 
        for e in section_names]

    # just split each file into text of it's sections
    out = {}

    # get lines associated with section
    def _get_section_text(section_name = 'AASEQ'):
        idx = section_names.index(section_name)
        section_text = r_text_list[section_starts[idx]:section_starts[idx+1]]
        # remove leading indent
        section_text = [e[12:] for e in section_text]
        return(section_text)

    for section_name in section_names:
        if section_name != '///': # end of file
            out[section_name] = _get_section_text(section_name = section_name)

    return(out)

In [ ]:
save_dir = '../data/zma/kegg/'
save_path = save_dir+'cached_kegg_gene_files.pkl'

ensure_dir_path_exists(dir_path = save_dir)

if not os.path.exists(save_path):
    # This is a pain. I would like to parallelize reading these files in, but since it only has to be done once per species doing so would be premature optimization.
    # BUT it seems that nbdev is rerunning it as part of the CI workflow. So caching it makes sense. 
    kegg_gene_files = os.listdir('../ext_data/zma/kegg/gene_entries/')
#     kegg_gene_entries = [kegg_gene_to_dict(kegg_file = kegg_gene_file) for kegg_gene_file in tqdm.tqdm(kegg_gene_files)]
    kegg_gene_entries = [kegg_gene_to_dict(kegg_file = kegg_gene_file) for kegg_gene_file in kegg_gene_files]
    # 37712/37712 [07:12<00:00, 87.23it/s]

    with open(save_path, 'wb') as handle:
        pkl.dump(kegg_gene_entries, 
                 handle, 
                 protocol=pkl.HIGHEST_PROTOCOL)
else:
    # Reading in data
    with open(save_path, 'rb') as handle:
        kegg_gene_entries = pkl.load(handle)

## Clean up KEGG gene dictionaries

The goal here is to use a finite-state machine to convert the lines for each section into a useful representation. This approach makes extending or altering this code easier.

Starting out I needed to find how many states were featured across all files.

In [ ]:
# produce a flat list of keys then deduplicate them
kegg_gene_sections = [entry for sublist in [list(kegg_gene_entries[i].keys()) for i in range(len(kegg_gene_entries))] 
                  for entry in sublist] # entry is defined here so without it the list comprehension fails instead of producing the list of sublists
kegg_gene_sections = list(set(kegg_gene_sections))
kegg_gene_sections 

['MOTIF',
 'ORGANISM',
 'AASEQ',
 'ENTRY',
 'NTSEQ',
 'NAME',
 'MODULE',
 'PATHWAY',
 'POSITION',
 'SYMBOL',
 'BRITE',
 'ORTHOLOGY',
 'STRUCTURE',
 'DBLINKS']

To aid in writing the logic for each section, pull several examples as a reference.

In [ ]:
# Helper Functions ---------------------------------------------------------------------
# helper fcn to pull indices and examples of sections
def get_section_examples(section = 'MOTIF', n = 5):
    i_section_matches = [i for i in range(len(kegg_gene_entries)) if section in kegg_gene_entries[i].keys()]
    out = [[i_section_matches[i], 
            kegg_gene_entries[i_section_matches[i]][section] ] for i in range(n)]
    return(out)

In [ ]:
get_section_examples(section = 'SYMBOL', n = 5)

[[6, ['z1C1_20']],
 [34, ['lox2']],
 [35, ['lox4']],
 [36, ['lox5']],
 [37, ['lox6']]]

Now create a function to process each section of the file or more precisely, each section type that needs a different treatment. 

In [ ]:
# Any non-hierarical list of attributes can go here. It will be transformed into a dict
def _gene_entry_flat_list(section_entry):
    # split like so [['NCBI-GeneID', '103644366'], ['NCBI-ProteinID', 'XP_020400304']]
    # then convert to dict
    # {'NCBI-GeneID': '103644366', 'NCBI-ProteinID': 'XP_020400304'}
    section_entry = [e.replace(section_name, '').strip().split(': ') for e in section_entry]
    section_entry = dict(section_entry)
    return(section_entry)

In [ ]:
# 'AASEQ', 'NTSEQ'
def _gene_seq_to_dict(seq_list):
    out = {}
    out['lenght'] = int(seq_list[0])
    out['seq'] = ''.join(seq_list[1:])
    return(out)

The hardest one to think through was the "BRITE". This section contains a tree categorizing the gene's role(s). Here's a sample excerpt:
```
KEGG Orthology (KO) [BR:zma00001]
 09120 Genetic Information Processing
  09121 Transcription
   03020 RNA polymerase
    100037782
 09180 Brite Hierarchies
  09182 Protein families: genetic information processing
   03021 Transcription machinery [BR:zma03021]
    100037782
# ...
```
The goal is to represent these data in a way that can be used to construct a hierarchy for all genes that are in a given sample. Level is defined by leading whitespace. 
```
KEGG Orthology (KO) [BR:zma00001]
├ 09120 Genetic Information Processing
│ └ 09121 Transcription
│   └ 03020 RNA polymerase
│     └ 100037782
└ 09180 Brite Hierarchies
  └ 09182 Protein families: genetic information processing
    └ 03021 Transcription machinery [BR:zma03021]
      └ 100037782
```
Parsing into a dictionary makes sense (and then creating a full hierarchy would be as simple as merging on keys) but building one is tricky to do since you can't pass a list of keys into a dictionary. Instead of reading from the root to the leaf, we work backwards to find the path from leaf to root. This makes finding the right parent easy -- track the indent level and go up one line skipping lines with an indent level >= the current level. These paths have redundant information (shared parents) but represent what is needed.

In [ ]:
# Figuring out how to process BRITE entries
kegg_gene_entry = kegg_gene_entries[1]
section_entry = kegg_gene_entry['BRITE']

section_entry

['KEGG Orthology (KO) [BR:zma00001]',
 ' 09180 Brite Hierarchies',
 '  09182 Protein families: genetic information processing',
 '   03000 Transcription factors [BR:zma03000]',
 '    100037740',
 'Transcription factors [BR:zma03000]',
 ' Eukaryotic type',
 '  Helix-turn-helix',
 '   Tryptophan clusters Myb, Myb-factors',
 '    100037740']

In [ ]:
# get the leading whitespace for each line
indent_spaces = [re.findall('^ +', e)[0] if re.match('^ ', e) else [''][0] for e in section_entry]
indent_spaces = [len(e) for e  in indent_spaces]

len(indent_spaces), indent_spaces

(10, [0, 1, 2, 3, 4, 0, 1, 2, 3, 4])

Because the lists are ordered, given a position one can start at a leaf and work back up to the root by 
walking backwards and looking for the next point at which the indent level is the expected level. 
IF it's true that each of the leaves contain only an identifer number and no letters
then the process is to
1. Find all leaves
1. Walk backwards from all leaves
1. Add a list of lists with these paths (could be used in a tidy format)

Below the process of building up the path from leaf to root and the indents at each step (as a sanity check) are shown.

In [ ]:
i = 4 # position in list
indent = indent_spaces[i] # indent at that position

j = i
current_indent = indent
# work backawards to get the paths
j_backtrack      = [j]
indent_backtrack = [current_indent]

while indent_backtrack[-1] > 0:
    while current_indent != indent_backtrack[-1]-1:
        j = j-1
        current_indent = indent_spaces[j]

    j_backtrack.extend([j])
    indent_backtrack.extend([current_indent])

    # indent
    print(j_backtrack, indent_backtrack) # 

[4, 3] [4, 3]
[4, 3, 2] [4, 3, 2]
[4, 3, 2, 1] [4, 3, 2, 1]
[4, 3, 2, 1, 0] [4, 3, 2, 1, 0]


Wrap this in a function, then integrate into a processing function for BRITE

In [ ]:
def _indent_backtrack_path(i, # position in list 
                           indent_spaces                                        
                          ):
    indent = indent_spaces[i] # indent at that position
    j = i
    current_indent = indent
    # work backawards to get the paths
    j_backtrack      = [j]
    indent_backtrack = [current_indent]

    while indent_backtrack[-1] > 0:
        while current_indent != indent_backtrack[-1]-1:
            j = j-1
            current_indent = indent_spaces[j]

        j_backtrack.extend([j])
        indent_backtrack.extend([current_indent])
        
    # indent_backtrack is not needed beyond debugging
    # confirm that the indent only decreases as you walk through the backtrack
    indent_check = indent_backtrack[1:]+[-1]
    indent_check =  [True if indent_backtrack[i] > indent_check[i] else False for i in range(len(indent_backtrack))]
    assert False not in indent_check
    
    return(j_backtrack)

In [ ]:
def _gene_entry_BRITE(section_entry):
    # get the leading whitespace for each line
    indent_spaces = [re.findall('^ +', e)[0] if re.match('^ ', e) else [''][0] for e in section_entry]
    indent_spaces = [len(e) for e  in indent_spaces]

    # find the leaves
    # ['KEGG Orthology (KO) [BR:zma00001]',
    #  ' 09120 Genetic Information Processing',
    #  '  09121 Transcription',
    #  '   03020 RNA polymerase',
    #  '    100037782' <------------- Whitespace followed by digits and no letters
    leaf_idxs = [i for i in range(len(section_entry)) if re.match('^\s+\d+$', section_entry[i])]

    # for each leaf get the backtrack path to the root
    leaf_backtraces = [_indent_backtrack_path(i = leaf_idx, 
                                              indent_spaces = indent_spaces
                                             ) for leaf_idx in leaf_idxs]
    # reverse the inner lists
    [e.reverse()  for e in leaf_backtraces]

    out = [[section_entry[i] for i in leaf_backtraces[j]] for j in range(len(leaf_backtraces))]
    # sample input
    # ['KEGG Orthology (KO) [BR:zma00001]',
    #  ' 09120 Genetic Information Processing',
    #  '  09121 Transcription',
    #  '   03020 RNA polymerase',
    #  '    100037782',
    #  ' 09180 Brite Hierarchies',
    #  '  09182 Protein families: genetic information processing',
    #  '   03021 Transcription machinery [BR:zma03021]',
    #  '    100037782',
    #  'Enzymes [BR:zma01000]',
    #  ' 2. Transferases',
    #  '  2.7  Transferring phosphorus-containing groups',
    #  '   2.7.7  Nucleotidyltransferases',
    #  '    2.7.7.6  DNA-directed RNA polymerase',
    #  '     100037782',
    #  'Transcription machinery [BR:zma03021]',
    #  ' Eukaryotic type',
    #  '  RNA polymerase II system',
    #  '   RNA polymerase II',
    #  '    Pol IV and V specific subunits',
    #  '     100037782']

    # Sample output
    # [['KEGG Orthology (KO) [BR:zma00001]',
    #   ' 09120 Genetic Information Processing',
    #   '  09121 Transcription',
    #   '   03020 RNA polymerase',
    #   '    100037782'],
    #  ['KEGG Orthology (KO) [BR:zma00001]',
    #   ' 09180 Brite Hierarchies',
    #   '  09182 Protein families: genetic information processing',
    #   '   03021 Transcription machinery [BR:zma03021]',
    #   '    100037782'],
    #  ['Enzymes [BR:zma01000]',
    #   ' 2. Transferases',
    #   '  2.7  Transferring phosphorus-containing groups',
    #   '   2.7.7  Nucleotidyltransferases',
    #   '    2.7.7.6  DNA-directed RNA polymerase',
    #   '     100037782'],
    #  ['Transcription machinery [BR:zma03021]',
    #   ' Eukaryotic type',
    #   '  RNA polymerase II system',
    #   '   RNA polymerase II',
    #   '    Pol IV and V specific subunits',
    #   '     100037782']]

    # and then strip whitespace
    out = [[ee.strip() for ee in e] for e in out]
    return(out)

In [ ]:
def parse_kegg_gene_entry(kegg_gene_entry):
    for section in kegg_gene_entry.keys():
        if type(kegg_gene_entry[section]) != list :
            # This is a safeguard to prevent the code from breaking when rerun. All entries start out as list
            # so if an entry isn't a list then it's already been transformed
            pass

        else:
            if section in ['TEMPLATE']:
                kegg_gene_entry[section] = kegg_gene_entry[section]

            elif section in ['ENTRY', 'NAME', 'ORTHOLOGY', 'ORGANISM', 'POSITION', 'SYMBOL', 'STRUCTURE']:
                kegg_gene_entry[section] = kegg_gene_entry[section][0]

            elif section in ['PATHWAY', 'MODULE']: 
                # NOTE: PATHWAY contains two spaces between the identifier and name which is why I'm not splitting on the first instance of whitespace.
                # 'zma00591  Linoleic acid metabolism'
                #          ^^        ^    ^
                kegg_gene_entry[section] = dict([e.split('  ') for e in kegg_gene_entry[section]])            

            elif section in ['BRITE']:
                # This dict is only expected to have a single key. Representing these data as a dict allows for the 
                # list checking logic above to still work.
                kegg_gene_entry[section] = {"BRITE_PATHS": _gene_entry_BRITE(section_entry = kegg_gene_entry[section])}

#             elif section in ['MODULE']:
#                 pass
#                 kegg_gene_entry[section] = _gene_entry_flat_list(section_entry=kegg_gene_entry[section])
                
            elif section in ['MOTIF', 'DBLINKS']:
                kegg_gene_entry[section] = _gene_entry_flat_list(section_entry=kegg_gene_entry[section])

            elif section in ['AASEQ', 'NTSEQ']:
                kegg_gene_entry[section] = _gene_seq_to_dict(seq_list = kegg_gene_entry[section])

            else:
                print("No behavior defined for "+section)

    return(kegg_gene_entry)

In [ ]:
parsed_kegg_gene_entries = [parse_kegg_gene_entry(kegg_gene_entry = kegg_gene_entry
#                                                  ) for kegg_gene_entry in tqdm.tqdm(kegg_gene_entries)]
                                                 ) for kegg_gene_entry in kegg_gene_entries]

## Write out cleaned data

In [ ]:
save_dir = '../data/zma/kegg/'

ensure_dir_path_exists(dir_path = '../data/zma/kegg/')

In [ ]:
with open(save_dir+'kegg_gene_entries.pkl', 'wb') as handle:
    pkl.dump(parsed_kegg_gene_entries, 
             handle, 
             protocol=pkl.HIGHEST_PROTOCOL)
    
# Reading in data
# with open('./data/kegg_gene_entries.pkl', 'rb') as handle:
#     kegg_gene_entries = pkl.load(handle)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()